In [1]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
import os
import pandas as pd

# Define the directory containing the main folders
main_directory = r'C:\Users\ctr22\Downloads\RAMPatientQuestions'

# Create a list of acceptable sheet names
acceptable_sheet_names = ['Service Numbers', 'Service Number', 'Service Number ', 'Service Number Summary']

# Create the main DataFrame
master_main = pd.DataFrame(columns=['Cell1', 'Cell2', 'Sum', 'Sum2'])

# Dictionary to store sums of values for each subfolder and keyword
keywords = [
    "Composite", "Extraction", "Debridement", "X-Ray", "Cataracts", 
    "Diabetes", "Vaccine" or "Vaccination", "Readers", "Single Vision Glasses" or "Mail" or
    "Mailing", "Bifocal"
]
keyword_sums = {keyword: {} for keyword in keywords}

def update_master_df(master_df, subfolder_df):
    for _, row in subfolder_df.iterrows():
        cell1, cell2, cell3, cell4 = row['Cell1'], row['Cell2'], row['Sum'], row['Sum2']
        match = master_df[(master_df['Cell1'] == cell1) & (master_df['Cell2'] == cell2)]
        
        if not match.empty:
            master_index = match.index[0]
            master_df.at[master_index, 'Sum'] += cell3
            master_df.at[master_index, 'Sum2'] += cell4
            # Update the new columns from subfolder_df
            for col in subfolder_df.columns:
                if col not in ['Cell1', 'Cell2', 'Sum', 'Sum2']:
                    master_df.at[master_index, col] = row[col]
        else:
            new_row = pd.DataFrame(row).transpose()
            master_df = pd.concat([master_df, new_row], ignore_index=True)
    
    return master_df

# Function to process each trial sheet
def process_trial_sheet(trial_path, master_df, subfolder_name):
    trial = None
    for sheet_name in acceptable_sheet_names:
        try:
            # Try to read the specified sheet, treating "N/A" and "None" as valid text
            trial = pd.read_excel(trial_path, sheet_name=sheet_name, keep_default_na=False, na_values=[])
            break  # Exit the loop if the sheet is found
        except ValueError:
            continue  # Try the next sheet name if the current one is not found

    if trial is None:
        print(f"No acceptable sheet found in '{trial_path}'. Skipping file.")
        return master_df  # Return the unchanged master_df if no valid sheet is found

    if trial.empty:
        print(f"No data found in '{trial_path}'. Skipping processing.")
        return master_df  # Return the unchanged master_df if the sheet is empty
    
    if trial.shape[1] < 4:
        print(f"Error: '{trial_path}' does not have enough columns (need at least 4).")
        return master_df  # Return the unchanged master_df if the sheet doesn't have enough columns
    
    # Sum the values in column 3 where column 2 contains the specified keywords
    for keyword in keyword_sums.keys():
        keyword_sum = trial[trial.iloc[:, 1].str.contains(keyword, na=False)].iloc[:, 2].sum()
        if subfolder_name in keyword_sums[keyword]:
            keyword_sums[keyword][subfolder_name] += keyword_sum
        else:
            keyword_sums[keyword][subfolder_name] = keyword_sum
    
    # Create column names for cell3 and cell4 based on the filename
    column3 = f"{os.path.basename(trial_path)}_Cell3"
    column4 = f"{os.path.basename(trial_path)}_Cell4"
    
    for index, trial_row in trial.iterrows():
        if len(trial_row) < 4:
            print(f"Skipping row in '{trial_path}': Insufficient columns.")
            continue
        
        cell1, cell2, cell3, cell4 = trial_row[0], trial_row[1], trial_row[2], trial_row[3]
        
        # Check if a matching row exists in master_df based on Cell1 and Cell2
        match = master_df[(master_df['Cell1'] == cell1) & (master_df['Cell2'] == cell2)]
        
        if not match.empty:
            # Update existing row in master_df
            main_index = match.index[0]
            master_df.at[main_index, column3] = cell3
            master_df.at[main_index, column4] = cell4
        else:
            # Append new row to master_df
            new_row = pd.DataFrame({
                'Cell1': [cell1],
                'Cell2': [cell2],
                column3: [cell3],
                column4: [cell4]
            })
            master_df = pd.concat([master_df, new_row], ignore_index=True)
    
    return master_df

# Function to iterate through main directory and process subfolders
def process_main_directory(main_directory):
    global master_main  # Declare master_main as global to modify it within the function
    if not os.path.isdir(main_directory):
        print(f"The directory {main_directory} does not exist.")
        return
    
    # Create a Pandas Excel writer using XlsxWriter as the engine
    writer = pd.ExcelWriter(os.path.join(main_directory, 'master_services.xlsx'), engine='xlsxwriter')
    
    for subdir in os.listdir(main_directory):
        subfolder_path = os.path.join(main_directory, subdir)
        if os.path.isdir(subfolder_path):
            main_df = pd.DataFrame(columns=['Cell1', 'Cell2', 'Sum', 'Sum2'])  # Create a new main DataFrame for each subfolder
            for filename in os.listdir(subfolder_path):
                if filename.endswith('.xlsx'):
                    trial_path = os.path.join(subfolder_path, filename)
                    main_df = process_trial_sheet(trial_path, main_df, subdir)
            
            main_df.fillna(0, inplace=True)  # Fill NaNs in main_df with 0 for numeric columns
            main_df['C'] = 0
            main_df['D'] = 0
            for index, row in main_df.iterrows():
                even_sum = 0
                odd_sum = 0
                for col_index in range(2, len(row)):
                    try:
                        value = float(row.iloc[col_index])
                    except ValueError:
                        value = 0
                    if (col_index - 2) % 2 == 0:
                        even_sum += value
                    else:
                        odd_sum += value
                main_df.at[index, 'C'] = even_sum
                main_df.at[index, 'D'] = odd_sum
            
            main_df = main_df[['Cell1', 'Cell2', 'C', 'D'] + [col for col in main_df.columns if col not in ['Cell1', 'Cell2', 'C', 'D']]]
            main_df = main_df[main_df['C'] != 0]
            main_df.sort_values(by=['Cell1', 'Cell2'], inplace=True)  # Sort by Cell1 and then by Cell2
            
            # Write each dataframe to a different worksheet
            main_df.to_excel(writer, sheet_name=subdir, index=False)
            print(f"Processed and added worksheet for subfolder '{subdir}'")
            
            # Update master_main DataFrame
            master_main = update_master_df(master_main, main_df)
    
    # Fill NaNs in master_main with 0 for numeric columns
    master_main.fillna(0, inplace=True)
    
    # Calculate columns C and D for master_main
    master_main['C'] = 0
    master_main['D'] = 0
    for index, row in master_main.iterrows():
        even_sum = 0
        odd_sum = 0
        for col_index in range(4, len(row)):  # Start from index 4 to skip Cell1, Cell2, Sum, and Sum2
            try:
                value = float(row.iloc[col_index])
            except ValueError:
                value = 0
            if (col_index - 4) % 2 == 0:
                even_sum += value
            else:
                odd_sum += value
        master_main.at[index, 'C'] = even_sum
        master_main.at[index, 'D'] = odd_sum
    
    master_main = master_main[['Cell1', 'Cell2', 'Sum', 'Sum2', 'C', 'D'] + [col for col in master_main.columns if col not in ['Cell1', 'Cell2', 'Sum', 'Sum2', 'C', 'D']]]
    master_main = master_main[master_main['C'] != 0]
    master_main.sort_values(by=['Cell1', 'Cell2'], inplace=True)  # Sort by Cell1 and then by Cell2
    
    # Write master DataFrame to a separate worksheet named 'Master' in the same Excel file
    master_main.to_excel(writer, sheet_name='Master', index=False)
    print(f"Added master worksheet to the Excel file.")
    
    # Create a single worksheet for all keyword charts
    add_charts_to_excel(writer, keyword_sums)
    
    # Close the Pandas Excel writer and save the Excel file
    writer.close()
    print(f"The master file has been saved as {os.path.join(main_directory, 'master_services.xlsx')}")

# Function to add charts to the Excel sheet
def add_charts_to_excel(writer, keyword_sums):
    # Create a worksheet for the charts
    workbook = writer.book
    chart_worksheet = workbook.add_worksheet('Keyword Charts')
    
    row = 0
    for keyword, sums in keyword_sums.items():
        keyword_df = pd.DataFrame(list(sums.items()), columns=['Subfolder', 'Sum'])
        
        # Write the data to the worksheet
        start_row = row
        keyword_df.to_excel(writer, sheet_name='Keyword Charts', startrow=start_row, index=False)
        row += len(keyword_df) + 2
        
        # Create a chart object
        chart = workbook.add_chart({'type': 'line'})
        
        # Configure the chart with data from the keyword worksheet
        chart.add_series({
            'name': keyword,
            'categories': ['Keyword Charts', start_row + 1, 0, start_row + len(keyword_df), 0],
            'values': ['Keyword Charts', start_row + 1, 1, start_row + len(keyword_df), 1],
        })
        
        # Add chart title and labels
        chart.set_title({'name': f'{keyword} Sums by Subfolder'})
        chart.set_x_axis({'name': 'Subfolder'})
        chart.set_y_axis({'name': 'Sum of Values'})
        
        # Insert the chart into the worksheet
        chart_worksheet.insert_chart(start_row, 3, chart)

# Process the main directory
process_main_directory(main_directory)


C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell1, cell2, cell3, cell4 = trial_row[0], trial_row[1], trial_row[2], trial_row[3]
C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell1, cell2, cell3, cell4 = trial_row[0], trial_row[1], trial_row[2], trial_row[3]
C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with 

Processed and added worksheet for subfolder '2016'


C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell1, cell2, cell3, cell4 = trial_row[0], trial_row[1], trial_row[2], trial_row[3]
C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell1, cell2, cell3, cell4 = trial_row[0], trial_row[1], trial_row[2], trial_row[3]
C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with 

Processed and added worksheet for subfolder '2017'
No acceptable sheet found in 'C:\Users\ctr22\Downloads\RAMPatientQuestions\2018\#918 One pager copy.xlsx'. Skipping file.


C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell1, cell2, cell3, cell4 = trial_row[0], trial_row[1], trial_row[2], trial_row[3]
C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell1, cell2, cell3, cell4 = trial_row[0], trial_row[1], trial_row[2], trial_row[3]
C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with 

...

Processed and added worksheet for subfolder '2023'


C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  master_df.at[master_index, col] = row[col]
C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  master_df.at[master_index, col] = row[col]
C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

Processed and added worksheet for subfolder '2024'


C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:151: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  master_main.fillna(0, inplace=True)
C:\Users\ctr22\AppData\Local\Temp\ipykernel_38632\2373404730.py:169: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '70.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  master_main.at[index, 'D'] = odd_sum


Added master worksheet to the Excel file.
The master file has been saved as C:\Users\ctr22\Downloads\RAMPatientQuestions\master_services.xlsx
